In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
############################################################
#1 - IMPORTAR AS TABELAS PARA TRABALHAR
############################################################

#importar o arquivo de treino

enem_train = pd.read_csv('C:/Users/oi400241/Desktop/Rafael/Curso Data Science/04.Codenation/train.csv')
enem_train.head()

#importar o arquivo de teste

enem_test = pd.read_csv('C:/Users/oi400241/Desktop/Rafael/Curso Data Science/04.Codenation/test.csv')
enem_test.head()


,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,41,PR,22,F,3,1,1,5,1,...,40.0,420.0,B,A,C,A,A,C,C,A
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,21,MA,26,F,3,1,1,8,1,...,100.0,580.0,E,B,C,B,B,B,F,A
2,b38a03232f43b11c9d0788abaf060f7366053b6d,23,CE,21,M,1,1,2,0,2,...,80.0,320.0,E,E,D,B,B,A,NaN,A
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,15,PA,27,F,3,1,1,8,1,...,NaN,NaN,H,E,G,B,B,A,NaN,A
4,715494628a50142ce8cb17191cfe6d0f3cae0934,41,PR,18,M,1,1,2,0,2,...,0.0,320.0,D,H,H,C,B,A,NaN,A


In [14]:
############################################################
#2 - TRATAMENTO DAS TABELAS
############################################################

#Script para normalizar as colunas
#1 - Validações
target = 'NU_NOTA_MT'
print('\nTrain and Test Datasets have the same columns?:',
      enem_train.drop(target,axis=1).columns.tolist()==enem_test.columns.tolist())
print("\nVariables not in test but in train : ", 
      set(enem_train.drop(target,axis=1).columns).difference(set(enem_test.columns)))

#2 - Remover as colunas
dif = list(set(enem_train.drop(target,axis=1).columns).difference(set(enem_test.columns)))

enem_train.drop(dif,axis=1,inplace=True)


Train and Test Datasets have the same columns?: True

Variables not in test but in train :  set()


In [17]:
from sklearn.ensemble import RandomForestRegressor

In [44]:
modelo = RandomForestRegressor(n_estimators = 9000, n_jobs = -1, random_state = 0)

In [52]:
variaveis = ['NU_NOTA_CN', 'NU_NOTA_LC', 'NU_NOTA_CH', 'NU_NOTA_REDACAO', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']

In [53]:
X = enem_train[variaveis]
y = enem_train['NU_NOTA_MT']

In [54]:
X = X.fillna(-1)

In [55]:
enem_train['NU_NOTA_MT'].isnull().sum()

0

In [35]:
enem_train["NU_NOTA_MT"].fillna(-1, inplace = True) 

In [56]:
#treinar o modelo
modelo.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=9000,
                      n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [57]:
#Aplicando no modelo de teste
X_prev = enem_test[variaveis]
X_prev = X_prev.fillna(-1)

In [40]:
X_prev.head()

,NU_NOTA_CN,NU_NOTA_LC,NU_NOTA_CH,NU_NOTA_REDACAO
0,464.8,431.8,443.5,420.0
1,391.1,548.0,491.1,580.0
2,595.9,613.6,622.7,320.0
3,-1.0,-1.0,-1.0,-1.0
4,592.9,571.4,492.6,320.0


In [58]:
p = modelo.predict(X_prev)

In [42]:
p

array([411.056, 438.138, 588.47 , ..., 676.332, 456.375,  -1.   ])

In [61]:
sub = pd.Series(p, index = enem_test['NU_INSCRICAO'], name = 'NU_NOTA_MT')
sub.shape

(4576,)

In [62]:
sub.to_csv("answer.csv", header = True)